<a href="https://colab.research.google.com/github/YuryAlvTap/Clase4_desafio_latam/blob/master/Pronosticos_online.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1 - Jornada y fechas

In [1]:
import bs4 as bs
import requests
from tqdm import tqdm
import pandas as pd

resp = requests.get("https://www.livefutbol.com/calendario/bundesliga-2023-2024-spieltag/3/") # "get" request to read data without making a change
resp = resp.text                       # fetch the HTML data
soup = bs.BeautifulSoup(resp, 'lxml')  # convert to BeautifulSoup-type object to work with HTML efficiently
                                       # lxml allows easy handling of HTML files/objects
table = soup.find('table', {'class': 'standard_tabelle'})  # 'wikitable sortable jquery-tablesorter' class does not appear
                                                                    # when navigating the website until the column is sorted,
                                                                    # so we should not use 'jquery-tablesorter'
fechas = []
horas = []
locales = []
versus =[]
visitas = []
resultados = []

for row in tqdm(table.findAll('tr')[0:]):
    fecha = row.findAll('td')[0].text
    hora = row.findAll('td')[1].text
    local = row.findAll('td')[2].text
    versu = row.findAll('td')[3].text
    visita = row.findAll('td')[4].text
    resultado = row.findAll('td')[5].text

    fechas.append(fecha)
    horas.append(hora)
    locales.append(local)
    versus.append(versu)
    visitas.append(visita)
    resultados.append(resultado)

df = pd.DataFrame(list(zip(fechas,horas,locales,versus,visitas,resultados)),
                  columns =['Fecha','Hora','Local','Versus','Visita','Resultado'])
df = df.reset_index(drop=True)
df

def clean_columns(x):
    return x.replace("\n", "")

def clean_foundation(x):
    return x.split('(')[-1].split(')')[0][-4:]

def get_state_country(x):
    return x.split(', ')[-1]

df["Fecha"] = df["Fecha"].apply(lambda x: clean_columns(x))
df["Hora"] = df["Hora"].apply(lambda x: clean_columns(x))
df["Local"] = df["Local"].apply(lambda x: clean_columns(x))
df["Versus"] = df["Versus"].replace('-', 'vs')
df["Visita"] = df["Visita"].apply(lambda x: clean_columns(x))
df["Resultado"] = df["Resultado"].apply(lambda x: clean_columns(x))
df

100%|██████████| 9/9 [00:00<00:00, 6681.19it/s]


,Fecha,Hora,Local,Versus,Visita,Resultado
0,01.09.2023,20:30,Borussia Dortmund,-,1. FC Heidenheim 1846,-:-
1,02.09.2023,15:30,Bayer Leverkusen,-,Darmstadt,-:-
2,,15:30,Hoffenheim,-,Wolfsburgo,-:-
3,,15:30,Werder Bremen,-,Mainz,-:-
4,,15:30,Augsburgo,-,VfL Bochum,-:-
5,,15:30,Stuttgart,-,SC Freiburg,-:-
6,,18:30,Bor. Mönchengladbach,-,Bayern Múnich,-:-
7,03.09.2023,15:30,Eintracht Frankfurt,-,Colonia,-:-
8,,17:30,1. FC Union Berlin,-,RB Leipzig,-:-


Tabla de posiciones y datos

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

web_url = "https://www.livefutbol.com/calendario/bundesliga-2023-2024-spieltag/3/"
#web_url = "https://www.livefutbol.com/calendario/eng-premier-league-2023-2024-spieltag/4/".
#web_url = "https://www.livefutbol.com/calendario/esp-primera-division-2023-2024-spieltag/4/"
data = requests.get(web_url).text

beautiful_soup = BeautifulSoup(data, 'html.parser')

print('Classes of Every table:')
for table in beautiful_soup.find_all('table',limit=5):
     table.get('class')
     print(table.get('class'))

mylist = []
for table_row in table.find_all('tr'):
    table_columns = table_row.find_all('td')
    if(table_columns != []):
        posicion = table_columns[0].text.strip()
        equipo = table_columns[2].text.strip()
        jugados = table_columns[3].text.strip()
        ganados = table_columns[4].text.strip()
        empatados = table_columns[5].text.strip()
        perdidos = table_columns[6].text.strip()
        goles = table_columns[7].text.strip()
        dif = table_columns[8].text.strip()
        puntos = table_columns[9].text.strip()

        mylist.append([posicion, equipo, jugados, ganados, empatados, perdidos, goles, dif, puntos])

df = pd.DataFrame(mylist,
     columns=['posicion', 'equipo', 'jugados', 'ganados', 'empatados', 'perdidos', 'goles', 'dif', 'puntos'])
df = df.reset_index(drop=True)
df

#print(df.head())

Classes of Every table:
['auswahlbox', 'with-border']
['standard_tabelle']
None
['auswahlbox']
['standard_tabelle']


,posicion,equipo,jugados,ganados,empatados,perdidos,goles,dif,puntos
0,1,1. FC Union Berlin,2,2,0,0,8:2,6,6
1,2,Bayern Múnich\n(M),2,2,0,0,7:1,6,6
2,3,Bayer Leverkusen,2,2,0,0,6:2,4,6
3,4,Wolfsburgo,2,2,0,0,4:1,3,6
4,5,SC Freiburg,2,2,0,0,3:1,2,6
5,6,Borussia Dortmund,2,1,1,0,2:1,1,4
6,,Eintracht Frankfurt,2,1,1,0,2:1,1,4
7,8,RB Leipzig\n(P),2,1,0,1,7:4,3,3
8,9,Stuttgart,2,1,0,1,6:5,1,3
9,10,Hoffenheim,2,1,0,1,4:4,0,3


Tabla posiciones de local

In [6]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

web_url = "https://www.livefutbol.com/calendario/esp-primera-division-2023-2024-spieltag/4/heim/"

data = requests.get(web_url).text

beautiful_soup = BeautifulSoup(data, 'html.parser')

print('Classes of Every table:')
for table in beautiful_soup.find_all('table',limit=1):
     table.get('class')
     print(table.get('class'))

mylist = []
for table_row in table.find_all('tr'):
    table_columns = table_row.find_all('td')
    if(table_columns != []):
        posicion = table_columns[0].text.strip()
        equipo = table_columns[2].text.strip()
        jugados = table_columns[3].text.strip()
        ganados = table_columns[4].text.strip()
        empatados = table_columns[5].text.strip()
        perdidos = table_columns[6].text.strip()
        goles = table_columns[7].text.strip()
        dif = table_columns[8].text.strip()
        puntos = table_columns[9].text.strip()

        mylist.append([posicion, equipo, jugados, ganados, empatados, perdidos, goles, dif, puntos])

df = pd.DataFrame(mylist,
     columns=['posicion', 'equipo', 'jugados', 'ganados', 'empatados', 'perdidos', 'goles', 'dif', 'puntos'])
df = df.reset_index(drop=True)
df

#print(df.head())

Classes of Every table:
['auswahlbox', 'with-border']


IndexError: ignored